In [9]:
import torch
import time
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.is_available()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# print("当前设备名称：", torch.cuda.get_device_name(device) if device.type == 'cuda' else 'CPU')

> 1、生成词索引构成的序列

In [10]:
batch_size = 2
max_num_src_words = 8
max_num_tgt_words = 8
model_dim = 8
max_src_seq_len = 5
max_tgt_seq_len = 5
max_position_len = 5
# src_len = torch.randint(2,5,(batch_size,))
# tgt_len = torch.randint(2,5,(batch_size,))
src_len = torch.Tensor([2, 4]).to(torch.int32)
tgt_len = torch.Tensor([4, 3]).to(torch.int32)
# 词索引构成的序列
src_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_src_words, (L,)), (0, max_src_seq_len-L)), 0) \
                     for L in src_len])
tgt_seq = torch.cat([torch.unsqueeze(F.pad(torch.randint(1, max_num_tgt_words, (L,)), (0, max_tgt_seq_len-L)), 0) \
                     for L in tgt_len])
print(src_seq)
print(tgt_seq)

tensor([[4, 1, 0, 0, 0],
        [5, 5, 7, 7, 0]])
tensor([[3, 2, 7, 4, 0],
        [6, 7, 7, 0, 0]])


> 2、采用Embedding的forward方法

In [11]:
# 实例化embedding # requires_grad 
# weight是随机的，训练时更新
src_embedding_table = nn.Embedding(max_num_src_words+1, model_dim)
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1, model_dim)
# print(src_embedding_table.weight, tgt_embedding_table.weight)
# Embedding存在 --call-- forward方法隐式
src_embedding = src_embedding_table(src_seq)
tgt_embedding_table = tgt_embedding_table(tgt_seq)
print(src_embedding)
# 由于为0 底下3排一样

tensor([[[-0.5355, -0.9739, -0.4599,  0.2081, -0.6606, -0.4836,  0.1033,
           0.9948],
         [-0.2193,  0.3768, -1.1767,  0.7611, -1.0107,  0.0720,  0.1343,
           0.0700],
         [ 0.3928, -0.1414, -0.7111, -1.4414, -0.6621,  1.3227, -0.2484,
          -0.3267],
         [ 0.3928, -0.1414, -0.7111, -1.4414, -0.6621,  1.3227, -0.2484,
          -0.3267],
         [ 0.3928, -0.1414, -0.7111, -1.4414, -0.6621,  1.3227, -0.2484,
          -0.3267]],

        [[ 1.7584, -1.2378, -0.6316,  0.6291,  0.5500, -2.1515, -2.3653,
           0.3652],
         [ 1.7584, -1.2378, -0.6316,  0.6291,  0.5500, -2.1515, -2.3653,
           0.3652],
         [ 2.1432, -0.0489,  0.1226,  0.3359, -0.6791,  2.3552,  0.4729,
           0.1686],
         [ 2.1432, -0.0489,  0.1226,  0.3359, -0.6791,  2.3552,  0.4729,
           0.1686],
         [ 0.3928, -0.1414, -0.7111, -1.4414, -0.6621,  1.3227, -0.2484,
          -0.3267]]], grad_fn=<EmbeddingBackward0>)


> 3、生成pos—embedding

In [12]:
# pos embedding    method1 
start_time = time.time()
pos_embedding = torch.zeros(max_position_len, model_dim)
for i in range(0, model_dim):
    for j in range(0, max_position_len):
        if i%2 == 0:
            # print(i,j)
            pos_embedding[j, i] = np.sin(j/np.power(10000, (i/model_dim)))
        else:
            pos_embedding[j, i] = np.cos(j/np.power(10000, (i/model_dim)))
# print(pos_embedding)
end_time = time.time()
# 计算执行时间并转换为微秒
execution_time_microseconds = (end_time - start_time) * 1e6
# 打印执行时间
print("np Time (microseconds):", execution_time_microseconds)
#///////////////////////////////////////////////////////////////////////////////////////////
# pos embedding    method2 
start_time = time.time()
pos_mat = torch.arange(max_position_len).reshape(-1, 1) 
i_mat = torch.pow(10000, torch.arange(0, 8, 2).reshape((1, -1))/model_dim)
i1_mat = torch.pow(10000, torch.arange(1, 8, 2).reshape((1, -1))/model_dim)
# print(pos_mat, i_mat, i1_mat)
pe_embedding_table = torch.zeros(max_position_len, model_dim)
pe_embedding_table[:, 0::2] = torch.sin(pos_mat/i_mat)
pe_embedding_table[:, 1::2] = torch.cos(pos_mat/i1_mat)
print(pe_embedding_table)
end_time = time.time()
# 计算执行时间并转换为微秒
execution_time_microseconds = (end_time - start_time) * 1e6
# 打印执行时间
print("torch Time (microseconds):", execution_time_microseconds)
#///////////////////////////////////////////////////////////////////////////////////////////
# print(torch.sub(pos_embedding, pe_embedding_table))
pe_embedding = nn.Embedding(max_position_len, model_dim)
# 位置编码训练中不发生变化requires_grad=False
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad=False)
# 注意 这里给embedding的是位置 不是词的编号
# src_p=torch.zeros_like(src_seq)
# for i in range(src_p.size(0)):
#     for j in range(src_p.size(1)):
#         src_p[i][j] = j 
# print(src_p)
# src_pe_embedding =pe_embedding(src_p)
src_pe_embedding =pe_embedding(torch.cat([pos_mat,pos_mat], 1).transpose(-1,-2))
print(torch.cat([pos_mat,pos_mat], 1).transpose(-1,-2))
# print(src_pe_embedding)
tgt_pe_embedding = src_pe_embedding

np Time (microseconds): 2277.1358489990234
tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  9.5042e-01,  9.9833e-02,  9.9950e-01,  9.9998e-03,
          9.9999e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01,  8.0658e-01,  1.9867e-01,  9.9800e-01,  1.9999e-02,
          9.9998e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01,  5.8275e-01,  2.9552e-01,  9.9550e-01,  2.9995e-02,
          9.9995e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01,  3.0114e-01,  3.8942e-01,  9.9201e-01,  3.9989e-02,
          9.9992e-01,  4.0000e-03,  1.0000e+00]])
torch Time (microseconds): 1752.8533935546875
tensor([[0, 1, 2, 3, 4],
        [0, 1, 2, 3, 4]])


> 4、生成enc_self_attn_mask 无因果关系   对词向量相似度进行masked

In [13]:
# 这里不是三角矩阵# 这里不是三角矩阵 没有因果关系 多用矩阵相乘 转置 升维
vaild_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len]), 2)
# print(vaild_encoder_pos.shape, "\n", vaild_encoder_pos)
# vaild_encoder_pos = torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len])
print(vaild_encoder_pos.shape, "\n", vaild_encoder_pos)
# 这里不是三角矩阵
# print(torch.tril(torch.ones(4, 4)))
# vaild_mask = vaild_encoder_pos.transpose(1,2)*torch.ones(4, 4)*vaild_encoder_pos
# print(vaild_mask.shape, "\n", vaild_mask)
# print(torch.bmm(torch.unsqueeze(torch.ones(4, 4),0), torch.unsqueeze(torch.ones(4, 4),0)))
# print(torch.ones(4, 4)*torch.ones(4, 4))
valid_encoder_pos_matrix = torch.bmm(vaild_encoder_pos, vaild_encoder_pos.transpose(1, 2))
invalid_encoder_pos_matrix = 1-valid_encoder_pos_matrix
mask_encoder_self_attention = invalid_encoder_pos_matrix.to(torch.bool)
print(mask_encoder_self_attention)
#假设是vk注意力权重
score = torch.randn(batch_size, max(src_len), max(src_len))
masked_score = score.masked_fill(mask_encoder_self_attention, -np.inf)
prob = F.softmax(masked_score, -1)
print(prob)

torch.Size([2, 4, 1]) 
 tensor([[[1.],
         [1.],
         [0.],
         [0.]],

        [[1.],
         [1.],
         [1.],
         [1.]]])
tensor([[[False, False,  True,  True],
         [False, False,  True,  True],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]],

        [[False, False, False, False],
         [False, False, False, False],
         [False, False, False, False],
         [False, False, False, False]]])
tensor([[[0.0146, 0.9854, 0.0000, 0.0000],
         [0.4324, 0.5676, 0.0000, 0.0000],
         [   nan,    nan,    nan,    nan],
         [   nan,    nan,    nan,    nan]],

        [[0.4723, 0.0236, 0.2912, 0.2130],
         [0.7549, 0.0775, 0.0501, 0.1174],
         [0.1201, 0.0977, 0.3192, 0.4629],
         [0.2615, 0.2745, 0.3395, 0.1246]]])


> 5、get_attn_pad_mask

In [14]:
enc_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(src_len)-L)),0) for L in src_len]), 1)
dec_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(tgt_len)-L)),0) for L in tgt_len]), 2)
print(enc_encoder_pos.shape, "\n", dec_encoder_pos.shape)
print(enc_encoder_pos, "\n", dec_encoder_pos)
# 具体维度通过【】的奇偶特征决定
get_attn_pad_mask = torch.bmm(dec_encoder_pos, enc_encoder_pos, )
print(get_attn_pad_mask, "\n", get_attn_pad_mask.shape)
score = torch.randn(2, max(src_len), max(tgt_len))
masked_score = score.masked_fill((1-get_attn_pad_mask).to(torch.bool), -np.inf)
prob = F.softmax(masked_score, -1)
print(prob)

torch.Size([2, 1, 4]) 
 torch.Size([2, 4, 1])
tensor([[[1., 1., 0., 0.]],

        [[1., 1., 1., 1.]]]) 
 tensor([[[1.],
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         [0.]]])
tensor([[[1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 0., 0.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [0., 0., 0., 0.]]]) 
 torch.Size([2, 4, 4])
tensor([[[0.5509, 0.4491, 0.0000, 0.0000],
         [0.5685, 0.4315, 0.0000, 0.0000],
         [0.8273, 0.1727, 0.0000, 0.0000],
         [0.8970, 0.1030, 0.0000, 0.0000]],

        [[0.6837, 0.1536, 0.0734, 0.0894],
         [0.0444, 0.1442, 0.2353, 0.5760],
         [0.2831, 0.0628, 0.6228, 0.0312],
         [   nan,    nan,    nan,    nan]]])


> 6、dec_self_attn_mask

In [15]:
tril = torch.unsqueeze(torch.tril(torch.ones(max(tgt_len), max(tgt_len))), 0)
print(tril.shape)
# tril = tril.expand(2, 4, 4) 
tril = torch.cat((tril, tril ))
print(tril)
dec_encoder_pos = torch.unsqueeze(torch.cat([torch.unsqueeze(F.pad(torch.ones(L),(0, max(tgt_len)-L)),0) for L in tgt_len]), 2)
valid_decoder_pos_matrix = torch.bmm(dec_encoder_pos,dec_encoder_pos.transpose(1, 2))
dec_self_attn_mask = valid_decoder_pos_matrix*tril
print(dec_self_attn_mask)

torch.Size([1, 4, 4])
tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]],

        [[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]]])
tensor([[[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [1., 1., 1., 1.]],

        [[1., 0., 0., 0.],
         [1., 1., 0., 0.],
         [1., 1., 1., 0.],
         [0., 0., 0., 0.]]])


> 7、loss

In [16]:
logits = torch.randn(2,3,4)
# batchsize=2 seqlen=3 vocal_size=4
label = torch.randint(0, 4, (2,3))
logits = logits.transpose(1,2)
print(F.cross_entropy(logits, label,reduction = 'none'))
print(F.cross_entropy(logits, label,ignore_index = 2, reduction = 'none'))
# print(F.cross_entropy(logits, label,ignore_index = 3))

tensor([[0.8926, 2.9831, 2.7514],
        [1.8626, 2.4915, 0.8839]])
tensor([[0.8926, 2.9831, 2.7514],
        [1.8626, 2.4915, 0.8839]])
